In [29]:
array = ['Dark Ocean', 'Paper camp', 'Paper camp', 'Dark Ocean', 'Modern day love', 
         'Modern day love', 'Future glitch', 'Paper camp', 'Paper camp']

def count_films(array):
    films = dict()
    for i in set(array):
        films[i] = array.count(i)
        
    return films

def is_equal(film, number, film_counts):
    if film_counts[film] == number:
        return True


film_counts = count_films(array)

print(is_equal('Paper camp', 4, film_counts))
        


True
